In [228]:
import sqlalchemy
import pandas as pd
import os
import dotenv
import json
import numpy as np
import csv
#import tablib
dotenv.load_dotenv('../.env')

True

In [36]:
#!pip install python_dotenv
#!pip install psycopg2-binary

In [37]:
user = os.environ['DB_USERNAME']
pw = os.environ['DB_PASSWORD']
host = os.environ['DB_HOST'] 
db_url = f'postgresql://{user}:{pw}@{host}:5432'
engine = sqlalchemy.create_engine(db_url)

In [265]:
def parse_csv_flipside(filepath):
    with open(filepath, newline='\n') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar="'")
        #for row in spamreader:
        #    print(', '.join(row))
        items = []
        rows = []
        for row in spamreader:
            rows.append(row)
            final_items = []
            temp_items = []
            num_opens = 0
            for i in row:
                if '{' in i:
                    #print ('open')
                    num_opens += 1
                    temp_items.append(i)
                elif num_opens == 0:
                    # simply append
                    final_items.append(i)
                
                elif '}' in i:
                    temp_items.append(i)
                    num_opens -= 1
                    if num_opens == 0:
                        #print ('close')
                        final_items.append(','.join(temp_items))
                        temp_items = []                    
                elif num_opens > 0:
                    temp_items.append(i)
            items.append(final_items)  
            #print (len(row))
            #print (row)
    #df = pd.DataFrame.from_records(items[1:], columns=items[0])
    #return df
    return items, rows

In [266]:
items, rows = parse_csv_flipside('../data/sudoswap_trades.csv')

In [267]:
sizes=[]
for i in items:
    sizes.append(len(i))
set(sizes)

{29, 30}

In [270]:
df = pd.DataFrame.from_records([i for i in items[1:] if len(i)==29], columns=items[0])
df.shape

(99999, 29)

In [271]:
df.head()

,BLOCK_NUMBER,BLOCK_TIMESTAMP,TX_HASH,EVENT_TYPE,PLATFORM_ADDRESS,PLATFORM_NAME,PLATFORM_EXCHANGE_VERSION,SELLER_ADDRESS,BUYER_ADDRESS,NFT_ADDRESS,...,PLATFORM_FEE,CREATOR_FEE,TOTAL_FEES_USD,PLATFORM_FEE_USD,CREATOR_FEE_USD,TX_FEE,TX_FEE_USD,ORIGIN_FROM_ADDRESS,ORIGIN_TO_ADDRESS,ORIGIN_FUNCTION_SIGNATURE
0,15402821,2022-08-24 12:05:29.000,0x528679d7330183d263cb4ce40e87c9e1df69a81182d3...,sale,0x2b2e8cda09bba9660dca5cb6233787738ad68329,sudoswap,pair router,0x82c84d9c3d125c6e3448e21b1e838c0448f04c2f,0xe48a1a831e2634bddd53550b711613b64ac1a4b3,0xf10a71e53398bf4e3cf5e2a0cad3c529ad19b834,...,0.0003011623341,0,null,null,0,0.001272257325,null,0xe48a1a831e2634bddd53550b711613b64ac1a4b3,0x2b2e8cda09bba9660dca5cb6233787738ad68329,0x11132000
1,15402817,2022-08-24 12:04:32.000,0x6f45cb2d4a84459da38ae55c606bb671dfbc7ad1a1e4...,sale,0x2b2e8cda09bba9660dca5cb6233787738ad68329,sudoswap,pair router,0x82c84d9c3d125c6e3448e21b1e838c0448f04c2f,0xab63b1925aeef953808094486e8c65c6eb537f85,0xf10a71e53398bf4e3cf5e2a0cad3c529ad19b834,...,0.0002999624842,0,null,null,0,0.001076106638,null,0xab63b1925aeef953808094486e8c65c6eb537f85,0x2b2e8cda09bba9660dca5cb6233787738ad68329,0x11132000
2,15402806,2022-08-24 12:02:43.000,0x4c1604e119618a01c4947117b4931f7580ae7a687424...,sale,0x2b2e8cda09bba9660dca5cb6233787738ad68329,sudoswap,pair router,0x3258128a48fc9db1886e953fcf4d70f67fb953c9,0x6cf9380f5e1a38e3b4c5101c1aa0211603fec13f,0xf10a71e53398bf4e3cf5e2a0cad3c529ad19b834,...,0.009049751244,0,null,null,0,0.001156168451,null,0x6cf9380f5e1a38e3b4c5101c1aa0211603fec13f,0x2b2e8cda09bba9660dca5cb6233787738ad68329,0x11132000
3,15402806,2022-08-24 12:02:43.000,0x3c9a01ffcf57cf3b34db3310c3463a1765da0ba4c2a2...,sale,0x2b2e8cda09bba9660dca5cb6233787738ad68329,sudoswap,pair router,0xf6f19d0c489b53f6d9b73523127679965f5e0afb,0xfdb634527b50da38dddd673059a25667bab7cd62,0x932261f9fc8da46c4a22e31b45c4de60623848bf,...,0.001431228702,0,null,null,0,0.000999888,null,0xfdb634527b50da38dddd673059a25667bab7cd62,0x2b2e8cda09bba9660dca5cb6233787738ad68329,0x11132000
4,15402801,2022-08-24 12:02:04.000,0x51f666e3f8f96f844f913f2739426538c9b1bb3d5ce0...,sale,0x2b2e8cda09bba9660dca5cb6233787738ad68329,sudoswap,pair router,0x82c84d9c3d125c6e3448e21b1e838c0448f04c2f,0x048f8073d361def400b483bcb4b7761d7be8507a,0xf10a71e53398bf4e3cf5e2a0cad3c529ad19b834,...,0.0002987674145,0,null,null,0,0.001630438447,null,0x048f8073d361def400b483bcb4b7761d7be8507a,0x2b2e8cda09bba9660dca5cb6233787738ad68329,0x11132000


In [279]:
# store df
df = df.drop_duplicates(subset=['TX_HASH','BLOCK_NUMBER'], keep='first')
df.to_sql('sudoswap_trades', engine, if_exists='replace')

826

In [274]:
df.TX_HASH.value_counts()

0xf87b06d588f3ea03865dd4f9d23974284caf466150169069fe4047d535c4748a    447
0x97cb60b0dad02d8ef2900f2cec041022faa4407dcb3e0a05e672d83f8cae1a3a    300
0xa454990b3ce85783a68b9ebeb4e610819e32ccecc4f7c311ca233ed764538e7b    216
0x95a0fe2e263f1dfee9af0bb020fc3236a5d4b9a900c49f89aceeffbffabcabdd    208
0x28acdcad8d9a1409b64b67006c1366cbaa8387155283bfe677f0a38cd913b88c    201
                                                                     ... 
0xaf97e0f88d1f2b281b02bb8772918d1a7081492df3e0dac5f7e1f6460f61918f      1
0x9f7ae32a5adb7f39a0f93c93e26b0610a51730e78bd6b4eef4cfb722ab884382      1
0xf21b12e15cede7f9f9b6d3aa892eb091a2e0d92026d1d1d4a04ff65ff65c9711      1
0x9b5dc411d9b34aa68d9e413c456839fe9b4c45b4424cf8ef6611295872561217      1
0x94993efc49b4d2b2713654da7e2e4cf1380e34609f63b1d9ff1a98466b999bda      1
Name: TX_HASH, Length: 41826, dtype: int64

In [278]:
df.drop_duplicates(subset=['TX_HASH','BLOCK_NUMBER'], keep='first').shape

(41826, 29)